# 01 Create Scope credentials

In [1]:
%%tsql -artifact Fabcon_database -type SQLDatabase
-- Create a master key for the database
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##')
begin
    create master key encryption by password = N'V3RYStr0NGP@ssw0rd!';
end


-- Create the database scoped credential for Azure OpenAI
if not exists(select * from sys.database_scoped_credentials where [name] = 'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/')
begin
    create database scoped credential [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/]
    with identity = 'HTTPEndpointHeaders', secret = '{"api-key":"YOUR_OPENAI_KEY"}';
end


# 02 Add Vector column In Product Table

In [2]:
%%tsql -artifact Fabcon_database -type SQLDatabase
alter table [SalesLT].[Product]
add  embeddings VECTOR(1536), chunk nvarchar(2000);

# 03 Create Embeddings Stored Procedure

In [3]:
%%tsql -artifact Fabcon_database -type SQLDatabase
create or alter procedure dbo.create_embeddings
(
    @input_text nvarchar(max),
    @embedding vector(1536) output
)
AS
BEGIN
declare @url varchar(max) = 'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-06-01';
declare @payload nvarchar(max) = json_object('input': @input_text);
declare @response nvarchar(max);
declare @retval int;

-- Call to Azure OpenAI to get the embedding of the search text
begin try
    exec @retval = sp_invoke_external_rest_endpoint
        @url = @url,
        @method = 'POST',
        @credential = [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/],
        @payload = @payload,
        @response = @response output;
end try
begin catch
    select 
        'SQL' as error_source, 
        error_number() as error_code,
        error_message() as error_message
    return;
end catch
if (@retval != 0) begin
    select 
        'OPENAI' as error_source, 
        json_value(@response, '$.result.error.code') as error_code,
        json_value(@response, '$.result.error.message') as error_message,
        @response as error_response
    return;
end
-- Parse the embedding returned by Azure OpenAI
declare @json_embedding nvarchar(max) = json_query(@response, '$.result.data[0].embedding');

-- Convert the JSON array to a vector and set return parameter
set @embedding = CAST(@json_embedding AS VECTOR(1536));
END;

#  04 Create Embedding for all products in product table

In [4]:
%%tsql -artifact Fabcon_database -type SQLDatabase
SET NOCOUNT ON
DROP TABLE IF EXISTS #MYTEMP 
DECLARE @ProductID int
declare @text nvarchar(max);
declare @vector vector(1536);
SELECT * INTO #MYTEMP FROM [SalesLT].Product
SELECT @ProductID = ProductID FROM #MYTEMP
SELECT TOP(1) @ProductID = ProductID FROM #MYTEMP
WHILE @@ROWCOUNT <> 0
BEGIN
    set @text = (SELECT p.Name + ' '+ ISNULL(p.Color,'No Color') + ' '+  c.Name + ' '+  m.Name + ' '+  ISNULL(d.Description,'')
                    FROM 
                    [SalesLT].[ProductCategory] c,
                    [SalesLT].[ProductModel] m,
                    [SalesLT].[Product] p
                    LEFT OUTER JOIN
                    [SalesLT].[vProductAndDescription] d
                    on p.ProductID = d.ProductID
                    and d.Culture = 'en'
                    where p.ProductCategoryID = c.ProductCategoryID
                    and p.ProductModelID = m.ProductModelID
                    and p.ProductID = @ProductID);
    exec dbo.create_embeddings @text, @vector output;
    update [SalesLT].[Product] set [embeddings] = @vector, [chunk] = @text where ProductID = @ProductID;
    DELETE FROM #MYTEMP WHERE ProductID = @ProductID
    SELECT TOP(1) @ProductID = ProductID FROM #MYTEMP
END

# 05 Create Find_product Stored procedure

In [6]:
%%tsql -artifact Fabcon_database -type SQLDatabase
create or alter procedure [dbo].[find_products]
@text nvarchar(max),
@top int = 10,
@min_similarity decimal(19,16) = 0.80
as
if (@text is null) return;
declare @retval int, @qv vector(1536);
exec @retval = dbo.create_embeddings @text, @qv output;
if (@retval != 0) return;
with vector_results as (
SELECT 
        p.Name as product_name,
        ISNULL(p.Color,'No Color') as product_color,
        c.Name as category_name,
        m.Name as model_name,
        d.Description as product_description,
        p.ListPrice as list_price,
        p.weight as product_weight,
        vector_distance('cosine', @qv, p.embeddings) AS distance
FROM
    [SalesLT].[Product] p,
    [SalesLT].[ProductCategory] c,
    [SalesLT].[ProductModel] m,
    [SalesLT].[vProductAndDescription] d
where p.ProductID = d.ProductID
and p.ProductCategoryID = c.ProductCategoryID
and p.ProductModelID = m.ProductModelID
and p.ProductID = d.ProductID
and d.Culture = 'en')
select TOP(@top) product_name, product_color, category_name, model_name, product_description, list_price, product_weight, distance
from vector_results
where (1-distance) > @min_similarity
order by    
    distance asc;

# 06 Change data types of returning results sets

In [7]:
%%tsql -artifact Fabcon_database -type SQLDatabase
-- #### ATTENTION: AI-generated code can include errors or operations you didn't intend. Review the code in this cell carefully before running it.

create or alter procedure [find_products_api]
    @text nvarchar(max)
    as 
    exec find_products @text
    with RESULT SETS
    (    
        (    
            product_name NVARCHAR(200),    
            product_color NVARCHAR(50),    
            category_name NVARCHAR(50),    
            model_name NVARCHAR(50),    
            product_description NVARCHAR(max),    
            list_price INT,    
            product_weight INT,    
            distance float    
        )
    )

In [8]:
%%tsql -artifact Fabcon_database -type SQLDatabase
exec find_products_api 'I am looking for a red bike'

# 07 Pass products details to Chat Endpoint

In [9]:
%%tsql -artifact Fabcon_database -type SQLDatabase
CREATE OR ALTER PROCEDURE [dbo].[prompt_answer]
@user_question nvarchar(max),
@products nvarchar(max),
@answer nvarchar(max) output

AS

declare @url nvarchar(4000) = N'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview';
declare @payload nvarchar(max) = N'{
    "messages": [
        {
            "role": "system",
            "content": "You are a sales assistant who helps customers find the right products for their question and activities."
        },
        {
            "role": "user",
            "content": "The products available are the following: ' + @products + '"
        },
        {
            "role": "user",
            "content": " ' + @user_question + '"
        }
    ]
}';

declare @ret int, @response nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint
    @url = @url,
    @method = 'POST', 
    @payload = @payload,
    @credential = [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/],    
    @timeout = 230,
    @response = @response output;

select json_value(@response, '$.result.choices[0].message.content');


# 08 Procedure to call chat endpoint

In [10]:
%%tsql -artifact Fabcon_database -type SQLDatabase
create or alter procedure [dbo].[find_products_chat]
@text nvarchar(max),
@top int = 3,
@min_similarity decimal(19,16) = 0.70
as
if (@text is null) return;
declare @retval int, @qv vector(1536), @products_json nvarchar(max), @answer nvarchar(max);
exec @retval = dbo.create_embeddings @text, @qv output;
if (@retval != 0) return;
with vector_results as (
SELECT 
        p.Name as product_name,
        ISNULL(p.Color,'No Color') as product_color,
        c.Name as category_name,
        m.Name as model_name,
        d.Description as product_description,
        p.ListPrice as list_price,
        p.weight as product_weight,
        vector_distance('cosine', @qv, p.embeddings) AS distance
FROM
    [SalesLT].[Product] p,
    [SalesLT].[ProductCategory] c,
    [SalesLT].[ProductModel] m,
    [SalesLT].[vProductAndDescription] d
where p.ProductID = d.ProductID
and p.ProductCategoryID = c.ProductCategoryID
and p.ProductModelID = m.ProductModelID
and p.ProductID = d.ProductID
and d.Culture = 'en')
select
top(@top)
@products_json = (STRING_AGG (CONVERT(NVARCHAR(max),CONCAT( 
                                product_name, ' ' ,
                                product_color, ' ',
                                category_name, ' ', 
                                model_name, ' ', 
                                product_description, ' ',
                                list_price, ' ',
                                product_weight )), CHAR(13)))
from vector_results
where (1-distance) > @min_similarity
group by distance
order by    
    distance asc;

set @products_json = (select REPLACE(REPLACE(@products_json, CHAR(13), ' , '), CHAR(10), ' , '));

exec [dbo].[prompt_answer] @text, @products_json, @answer output;


# 09 Call chat stored procedure

In [11]:
%%tsql -artifact Fabcon_database -type SQLDatabase
create or alter procedure [find_products_chat_api]
    @text nvarchar(max)
    as 
    exec find_products_chat @text
    with RESULT SETS
    (    
        (    
            answer NVARCHAR(max)
        )
    )

# Test Procedure

In [12]:
%%tsql -artifact Fabcon_database -type SQLDatabase
exec find_products_chat_api 'I am looking for a red bike'